## Lab 4: Image colour space for object segmentation

Written by: Enrique Mireles Gutiérrez  
ID Number: 513944  
Bachelor: ITR  
Date: 2019-02-19  

### Introduction

Object segmentation in images or video can be done by selecting the pixels with similar color. This can be useful in several applications. For instance, in autonomous vehicles, object segmentation could be used to identify traffic lights, traffic lanes, traffic signs, and many other objects; all of them uniquely identified by their color. 

The main difficulty when performing object segmentation is lighting. When lightning changes, colors can be perceived darker or lighter, therefore producing different shades, tints, and tones of the same hue. Normally, cameras provide images in the RGB color space (BGR for OpenCV). The main disadvantage of this color space is that lightning affects the three components of the color (red, green, blue), and therefore, it becomes difficult to select a range of colors programmatically. Another alternative must be used.

HSV is another color space that is composed of three variables:
- hue: pure color it resembles. All shades, tones, and tints of the of x color have the same hue.
- saturation: describes how white the color is. Pure red is fully saturated. Meanwhile, white has a saturation of 0.
- value: also called lightness, describes how dark the color is. A value of 0 means black.

In the HSV color space, when lighting changes, the hue component remains constant for the most part. This advantage makes it easy to select hues under different lighting conditions. For this reason, it is used as a robust alternative to the RGB color space for object segmentation.

### Objectives

In this lab, you will learn about image colour space, particularly about the HSV (Hue, Saturation, Value) and HLS (Hue, Saturation, Lightness) colour maps. You will use this colour space to segment an object in a video sequence, this object can be either a fish swimming in a fishbolw, lane lines on a road, or any other object that can be segmented by using its corresponding HSV components.

### Procedure

This lab report is subdivided in smaller numbered programs shown below.

#### 1. Importing Libraries

The following libraries are used throughout the lab report:
- cv2: OpenCV library used for artificial vision.
- numpy: Library used for matrix operations.
- time: Used to create delays to slow down the video playback.

In [3]:
import numpy as np
import cv2
import time

#### 2. Constant definitions

The following lines define the constants used throughout the lab report:

In [4]:
FISH_VIDEO = '../fig/videos/volti-01.mp4'
RUNNING_TRACK_VIDEO = '../fig/videos/running-track.mp4'
MARDIGRASS_VIDEO = '../fig/videos/mardigrass.mp4'

#### 3. Useful functions

The following functions are used several times. In order to avoid repetition they are defined here.

In [5]:
def inHSVRanges(image, ranges):
    """
        Given an array of ranges containing min and max hsv values,
        this function returns a mask that combines all ranges.
        It also displays every mask in its own window.
    """
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    result = None
    for r in ranges:
        mask = cv2.inRange(hsv, r[0], r[1])
        cv2.imshow(r[2], cv2.bitwise_and(image, image, mask=mask))
        if (result is None):
            result = mask
        else:
            result = cv2.bitwise_or(result, mask)
    return result

def createHSVRangeWindows(ranges):
    """
        For every range create its own window.
    """
    
    for r in ranges:
        cv2.namedWindow(r[2], cv2.WINDOW_NORMAL)

def resize(image, factor):
    """
        Returns a resized image by the specified factor.
    """
    
    new_h = int(image.shape[0] / factor)
    new_w = int(image.shape[1] / factor)
    return cv2.resize(image, (new_w, new_h)) 

def openVideo(source_video_file):
    """
        This function attempts to open a video file.
        If it fails, it exits the application.
    """
    
    cap = cv2.VideoCapture(source_video_file)
    if(not cap.isOpened()):
        print("Error opening video source")
        exit()
    return cap

def closeWindows(cap):
    """
        Release videos and destroy windows.
    """
    cap.release()
    cv2.destroyAllWindows()

#### 4. Types of color spaces available

This program prints the available color spaces in OpenCV.

In [ ]:
colour_space_list = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(colour_space_list)

#### 5. Object segmentation

This program obtains all pixels within a HSV color range in order to segement an object. The program can choose between 4 sample videos to segment. To change the video, modify the parameter `dataset_number` in the main function.

In [13]:
def processVideo(cap, hsv_min = (0, 0, 0), hsv_max = (180, 255, 255)):

    # Create new windows for visualisation purposes
    cv2.namedWindow('Input video', cv2.WINDOW_NORMAL)
    cv2.namedWindow('Segmented object', cv2.WINDOW_NORMAL)

    while(cap.isOpened()):

        # Grab current frame.
        ret, frame = cap.read()

        # Verify that frame was properly captured.
        if ret == False:
            print("ERROR: current frame could not be read")            
            break

        # Convert BGR to HSV.
        frame = resize(frame, 4)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Threshold the hsv image so that only blue pixels are kept.
        mask = cv2.inRange(hsv, hsv_min, hsv_max)

        # AND-bitwise operation between the mask and input images.
        segmented_objects = cv2.bitwise_and(frame, frame, mask=mask)

        # Visualise current frame.
        cv2.imshow('input video',frame)

        # Visualise segmented blue object.
        cv2.imshow('segmented object', segmented_objects)

        # Exit when the user presses the letter q.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def main(dataset_number = 1):
    
    # Define an array containing min hsv, max hsv and the video file.
    data = [
        [(115, 30, 0), (190, 250, 250), FISH_VIDEO],
        [(130, 0, 100), (160, 255, 255), RUNNING_TRACK_VIDEO],
        [(160, 85, 85), (180, 200, 200), RUNNING_TRACK_VIDEO],
        [(95, 40, 100), (110, 255, 255), MARDIGRASS_VIDEO]
    ]
    
    # Make sure dataset number is within a valid range.
    if (dataset_number >= 1 and dataset_number <= len(data)):
        
        # Open the selected video file.
        selected = data[dataset_number - 1]
        cap = openVideo(selected[2])
        
        # Filter video.
        processVideo(cap, hsv_min = selected[0], hsv_max = selected[1])
        
        # When video finishes, close windows.
        closeWindows(cap)
    
    else:
        
        print("Invalid dataset number.")
        exit()
    
main(dataset_number = 1)

#### 6. Segment multiple objects

In order to segment multiple objects, multiple ranges must be specified. The following two cells select the ranges to be displayed. Then the last cell uses those ranges to display the segmented objects.

![Segmentation of the fish and rocks in the fish video.](fish-fishbow-segmentation.png)
_Segmentation of the fish and rocks in the fish video._

![Segmentation of blue, red, yellow, and violet objects in the mardigrass video.](mardigrass-segmentation.png)
_Segmentation of blue, red, yellow, and violet objects in the mardigrass video._

In [53]:
"""
    Ranges to segment the fish and the fishbowl rocks.
"""

scale = 4
ranges = [
    [(90,  40, 0), (105, 250, 250), "rocks"],
    [(115, 30, 0), (190, 250, 250), "fish" ]
]
INPUT_VIDEO = FISH_VIDEO

In [54]:
"""
    Ranges to segment blue, red, yellow, and violet color objects.
"""

scale = 2
ranges = [
    [(0,   100, 0), (10,  250, 250),  "red"   ],
    [(20,  80,  0), (30,  250, 250),  "yellow"],
    [(95,  50,  50), (105, 250, 250),  "blue"  ],
    [(125, 30,  0), (135, 250, 250),  "violet"],
]
INPUT_VIDEO = MARDIGRASS_VIDEO

In [55]:
# Open the specified video file.
cap = openVideo(INPUT_VIDEO)

# Create windows for input, output objects, and all ranges.
cv2.namedWindow('input video', cv2.WINDOW_NORMAL)
cv2.namedWindow('segmented objects', cv2.WINDOW_NORMAL)
createHSVRangeWindows(ranges)

while(cap.isOpened()):
    
    # Get the current frame.
    ret, frame = cap.read()
    if ret == False:
        print("ERROR: current frame could not be read")
        break
    
    # Scale frame.
    frame = resize(frame, scale)
    
    # Create a mask containing all ranges. Also display
    # the range in its corresponding window.
    mask = inHSVRanges(frame, ranges)
    
    # Apply mask to input frame.
    segmented_objects = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Display results.
    cv2.imshow('input video', frame)
    cv2.imshow('segmented objects', segmented_objects)
    
    # If the user presses the letter q exit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    # Slow down video.
    time.sleep(1/30)

# Tidy up windows and exit.
closeWindows(cap)

### Conclusions

In conclusion, working in the RGB color space isn’t that robust as working in the HSV color space. The main advantage is presented when the lighting changes. In the RGB color space, when the amount of light varies, different shades of the same color are perceived and represented in three variables (red, green, and blue). These three variables can change completely even through they could represent the same hue under different light conditions. This situation doesn’t happen under the HSV color space. The hue is maintained under different lighting conditions. Therefore, when segmenting colors is a required task, the HSV color space is ideal.

### References

Shipman, J. (2012). 4.3. _The hue-saturation-value (HSV) color model._  
Retrieved from: http://infohost.nmt.edu/tcc/help/pubs/colortheory/web/hsv.html

_I hereby affirm that I have done this activity with academic integrity._